# Generate a simulation of a population by running envs_beta/BogoBetaEnv  environment

JMA 14 March 2023

See also `generate_primary_sim.py`

In [1]:
import math, os, re, sys
from pathlib import Path
import pandas as pd
import numpy as np

sys.path.append('../RL_offline/')
from beta_policies import BogoPolicies

/home/jma/repos/bogovirus/beta


In [6]:
### Use this policy for test

MAX_DOSE = 0.7 
VERBOSE = True

# def randomized_const_policy(yesterday, today):  # default policy
#     dose = BogoPolicies.my_rng.uniform(low=0, high=MAX_DOSE, size=1)[0]
#     if VERBOSE: print(f'chohort dose: {dose:.3}')
#     return dose
   

### a test run
def test_patient_run(env):
    ''
    for p in range(2):
        # Create a patient with a random Id. 
        the_patient = BogoPolicies.my_rng.integers(low=0, high=100000, size=1)[0]
        observation, info = env.reset(id_serial= the_patient)
        print('\t', the_patient)
        for _ in range(BogoPolicies.MAX_DAYS):
                observation, reward, terminated, info = env.step(env.standard_of_care_policy)
                if VERBOSE: 
                    print(env.today)
                else:
                    print(f'i: {info} # obs: {env.get_observation()}, R: {reward}, end? {terminated}' )
                if terminated:
                    break


bogo_env = BogoPolicies()
test_patient_run(bogo_env)
episode_df, total_reward = bogo_env.close()
print(f'DONE! - reward {total_reward}')
    


	 93555
{'patient_id': 93555, 'cohort': 0, 'day_number': 1, 'infection': 46, 'severity': 28.482948077399676, 'drug': 0.3, 'cum_drug': 0.08935597776639556, 'efficacy': 3.2986568120913358, 'outcome': None, 'reward': -1}
{'patient_id': 93555, 'cohort': 0, 'day_number': 2, 'infection': 53, 'severity': 33.25491007878928, 'drug': 0.6, 'cum_drug': 0.23421016639027198, 'efficacy': 6.89551169247591, 'outcome': None, 'reward': -1}
{'patient_id': 93555, 'cohort': 0, 'day_number': 3, 'infection': 54, 'severity': 34.24272485084991, 'drug': 0.6, 'cum_drug': 0.3412160253064296, 'efficacy': 7.080656415595466, 'outcome': None, 'reward': -1}
{'patient_id': 93555, 'cohort': 0, 'day_number': 4, 'infection': 54, 'severity': 36.94805055343261, 'drug': 0.6, 'cum_drug': 0.4210437076460055, 'efficacy': 5.907765193878943, 'outcome': None, 'reward': -1}
{'patient_id': 93555, 'cohort': 0, 'day_number': 5, 'infection': 54, 'severity': 41.767318898736775, 'drug': 0.6, 'cum_drug': 0.4782819039551828, 'efficacy': 4.7

In [9]:
bogo_env.NUM_COHORTS

1